In [ ]:
!pip install requests folium polyline geopy


In [ ]:
import folium
import polyline
import requests
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="manikandanreathi78@gmail.com", timeout=10)

def get_lat_lon(place_name):
    location = geolocator.geocode(place_name)
    if location:
        return location.latitude, location.longitude
    else:
        print(f"Could not find location for: {place_name}")
        return None

def get_route_data(start_location, end_location, api_key):
    url = f"https://graphhopper.com/api/1/route?point={start_location[0]},{start_location[1]}&point={end_location[0]},{end_location[1]}&vehicle=car&alternatives=3&key={api_key}"
    response = requests.get(url)

    if response.status_code == 200:

        print(response.json())
        return response.json()
    else:
        print(f"Error fetching route data: {response.status_code}")
        return None


def plot_route_on_map(start_location, end_location, route_data):

    m = folium.Map(location=[start_location[0], start_location[1]], zoom_start=10)


    for i, path in enumerate(route_data["paths"]):
        polyline_points = path["points"]
        decoded_points = polyline.decode(polyline_points)
        route_points = [(lat, lon) for lat, lon in decoded_points]

        route_color = f"#{hex(0x100000 * i)[2:].zfill(6)}"
        folium.PolyLine(route_points, color=route_color, weight=5, opacity=0.7, popup=f"Route {i+1}").add_to(m)


    folium.Marker(start_location, popup="Start").add_to(m)
    folium.Marker(end_location, popup="End").add_to(m)


    m.save("route_map.html")
    print("Map saved as route_map.html")

def get_user_input():
    print("Enter the start location (place name):")
    start_place = input("Start Place: ")
    print("Enter the end location (place name):")
    end_place = input("End Place: ")

    start_location = get_lat_lon(start_place)
    end_location = get_lat_lon(end_place)

    if start_location and end_location:
        return start_location, end_location
    else:
        return None, None

def main():
    api_key = "f60f177a-c980-48e0-8ce0-7530704a3d8c"
    start_location, end_location = get_user_input()

    if start_location and end_location:
        route_data = get_route_data(start_location, end_location, api_key)
        if route_data:
            print(f"Suggested Routes:")
            for i, path in enumerate(route_data["paths"]):
                print(f"Route {i+1}:")
                print(f"Distance: {path['distance'] / 1000} km")
                print(f"Estimated Travel Time: {path['time'] / 3600} hours")

            plot_route_on_map(start_location, end_location, route_data)
        else:
            print("Could not retrieve route data. Please check your input.")
    else:
        print("Could not find the locations. Please try again.")

if __name__ == "__main__":
    main()


Enter the start location (place name):
Start Place: Delhi 
Enter the end location (place name):
End Place: Tamilnadu
{'hints': {'visited_nodes.sum': 5276, 'visited_nodes.average': 5276.0}, 'info': {'copyrights': ['GraphHopper', 'OpenStreetMap contributors'], 'took': 42, 'road_data_timestamp': '2025-01-15T01:00:00Z'}, 'paths': [{'distance': 2381584.514, 'weight': 136003.604677, 'time': 100279851, 'transfers': 0, 'points_encoded': True, 'points_encoded_multiplier': 100000.0, 'bbox': [74.431743, 10.909963, 78.423752, 28.628504], 'points': '_kvmDwp_vM{@cDe@aBQo@Ik@VWl@e@s@kBE{BIuAGuCTErFmCLIzA{AtAy@bAu@JMdByDhBsCpCaEbAuAbBvCtA|Bv@lA~EvDbCrBnEpDNFn@Nx@FrAH~CF~ANvBd@|KhFjAXh@FfAArCOz@@vA?d@Bj@Lz@X`C`BnDzCxC`CnCbCjCfBhBn@nEz@dCz@bM`GjHlDZPtEzBpFrCj@HdIl@hBj@dA\\|@Pr@B`DCf@Cz@MpBqAZk@Jg@Bw@De@Ro@HKHIx@]fASTAT?TBRHRRPTFLHRDR@R?RCTERGRQVUTKHYL{Ax@s@\\MJKLKNIPERCRAR@h@Fr@R~@hBrEb@nAlF|PjBxF~BpHvC`LnAfEdBpFfJtZr@pBXj@xAbCvBbCdAfAnAbA^R`AXxCl@nZbF~Ev@zJnChB`A|CvCt@rAh@hBtCnJhCxGzDpI`BlCn@pAR^^f@b@f